## T. Kong, J. Shao, J. Hu, X. Yang, S. Yang, and R. Malekian, “EEG based emotion recognition using an improved weighted horizontal visibility graph,” Sensors, vol. 21, no. 5, p. 1870, 2021.


In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
path = '/content/drive/My Drive/PROJECT/EEG/emotions.csv'

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_size = 0.7
num_epochs = 5
batch_size = 16
learning_rate = 1e-3
num_classes = 3

# RNN model
input_size = 1
hidden_size = 256
num_layers = 2

In [ ]:
df = pd.read_csv(path)
df.head()

,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,...,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b,label
0,4.62,30.3,-356.0,15.6,26.3,1.070,0.411,-15.70,2.06,3.15,...,23.5,20.3,20.3,23.5,-215.0,280.00,-162.00,-162.00,280.00,NEGATIVE
1,28.80,33.1,32.0,25.8,22.8,6.550,1.680,2.88,3.83,-4.82,...,-23.3,-21.8,-21.8,-23.3,182.0,2.57,-31.60,-31.60,2.57,NEUTRAL
2,8.90,29.4,-416.0,16.7,23.7,79.900,3.360,90.20,89.90,2.03,...,462.0,-233.0,-233.0,462.0,-267.0,281.00,-148.00,-148.00,281.00,POSITIVE
3,14.90,31.6,-143.0,19.8,24.3,-0.584,-0.284,8.82,2.30,-1.97,...,299.0,-243.0,-243.0,299.0,132.0,-12.40,9.53,9.53,-12.40,POSITIVE
4,28.30,31.3,45.2,27.3,24.5,34.800,-5.790,3.06,41.40,5.52,...,12.0,38.1,38.1,12.0,119.0,-17.60,23.90,23.90,-17.60,NEUTRAL


In [ ]:
# Spliting X and y to train and test data
def split_data(X, y, train_size):
    train_size = int(len(X) * train_size)
    X_train = X[:train_size]
    y_train = y[:train_size]

    X_test = X[train_size:]
    y_test = y[train_size:]

    return X_train, X_test, y_train, y_test

In [ ]:
# Creating X and y and replacing labels
def preprcess_data(df, train_size=0.7):
    df = df.copy()

    y = df['label'].copy()
    y = y.replace(labels)

    X = df.drop('label', axis=1).copy()


    X_train, X_test, y_train, y_test = split_data(X, y, train_size)
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = preprcess_data(df, train_size)

<ipython-input-11-685e5a614455>:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace(labels)


In [ ]:
# Viewing X
X_train

,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,...,fft_740_b,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b
0,4.6200,30.3,-356.0,15.60,26.3,1.070,0.411,-15.7000,2.060,3.150,...,74.30,23.5,20.3,20.3,23.5,-215.0,280.00,-162.00,-162.00,280.00
1,28.8000,33.1,32.0,25.80,22.8,6.550,1.680,2.8800,3.830,-4.820,...,130.00,-23.3,-21.8,-21.8,-23.3,182.0,2.57,-31.60,-31.60,2.57
2,8.9000,29.4,-416.0,16.70,23.7,79.900,3.360,90.2000,89.900,2.030,...,-534.00,462.0,-233.0,-233.0,462.0,-267.0,281.00,-148.00,-148.00,281.00
3,14.9000,31.6,-143.0,19.80,24.3,-0.584,-0.284,8.8200,2.300,-1.970,...,-183.00,299.0,-243.0,-243.0,299.0,132.0,-12.40,9.53,9.53,-12.40
4,28.3000,31.3,45.2,27.30,24.5,34.800,-5.790,3.0600,41.400,5.520,...,114.00,12.0,38.1,38.1,12.0,119.0,-17.60,23.90,23.90,-17.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1487,9.1300,26.5,-231.0,6.53,26.4,6.230,3.190,9.7900,0.352,4.480,...,-652.00,508.0,-244.0,-244.0,508.0,-62.5,128.00,-51.60,-51.60,128.00
1488,26.1000,32.3,28.4,24.90,28.4,3.020,0.444,3.7100,2.720,-2.440,...,9.23,-41.0,-56.7,-56.7,-41.0,-10.6,-9.68,-138.00,-138.00,-9.68
1489,13.5000,31.1,-481.0,8.86,25.2,-1.050,-0.428,25.5000,2.030,0.315,...,-533.00,506.0,-252.0,-252.0,506.0,-444.0,461.00,-221.00,-221.00,461.00
1490,13.4000,18.3,-361.0,2.57,26.0,3.170,4.710,-0.0477,-0.202,-4.410,...,-289.00,284.0,-52.1,-52.1,284.0,-229.0,209.00,-61.50,-61.50,209.00


In [ ]:
class EEGBrainWavePreTrain(Dataset):
    def __init__(self, X):
        # x.shape: (N, 2548)
        self.X = torch.from_numpy(X.to_numpy()[:, :-1].astype(np.float32)) # (N, 2547)
        self.y = torch.from_numpy(X.to_numpy()[:, -1].astype(np.float32)) # (N)
        self.y = self.y.view(-1, 1) # (N, 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
class EEGBrainWaveFineTune(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.to_numpy().astype(np.float32))
        self.y = torch.from_numpy(y.to_numpy())

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
train_data = EEGBrainWavePreTrain(X_train)
test_data =  EEGBrainWavePreTrain(X_test)
train_data_fn = EEGBrainWaveFineTune(X_train, y_train)
test_data_fn =  EEGBrainWaveFineTune(X_test, y_test)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=1)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=1)

In [ ]:
train_loader_fn = DataLoader(train_data_fn, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=1)
test_loader_fn = DataLoader(test_data_fn, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=1)
X_train.shape

(1492, 2548)

In [ ]:
# for calculating the accuracy
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


class EEGClassificationBase(nn.Module):

    def training_step(self, batch):
        images, labels = batch
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss

    def validation_step(self, batch):
        images, labels = batch
        out = self(images)                   # Generate prediction
        loss = F.cross_entropy(out, labels)  # Calculate loss
        acc = accuracy(out, labels)          # Calculate accuracy
        return {"val_loss": loss.detach(), "val_accuracy": acc}

    def validation_epoch_end(self, outputs):
        batch_losses = [x["val_loss"] for x in outputs]
        batch_accuracy = [x["val_accuracy"] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()       # Combine loss
        epoch_accuracy = torch.stack(batch_accuracy).mean()
        return {"val_loss": epoch_loss, "val_accuracy": epoch_accuracy} # Combine accuracies

    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_accuracy']))

In [ ]:
# Architecture for training

# convolution block with BatchNormalization
def ConvBlock(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
             nn.BatchNorm2d(out_channels),
             nn.ReLU(inplace=True)]
    if pool:
        layers.append(nn.MaxPool2d(4))
    return nn.Sequential(*layers)


# resnet architecture
class ResNet9(EEGClassificationBase):
    def __init__(self, in_channels, num_diseases):
        super().__init__()

        self.conv1 = ConvBlock(in_channels, 64)
        self.conv2 = ConvBlock(64, 128, pool=True) # out_dim : 128 x 64 x 64
        self.res1 = nn.Sequential(ConvBlock(128, 128), ConvBlock(128, 128))

        self.conv3 = ConvBlock(128, 256, pool=True) # out_dim : 256 x 16 x 16
        self.conv4 = ConvBlock(256, 512, pool=True) # out_dim : 512 x 4 x 44
        self.res2 = nn.Sequential(ConvBlock(512, 512), ConvBlock(512, 512))

        self.classifier = nn.Sequential(nn.MaxPool2d(4),
                                       nn.Flatten(),
                                       nn.Linear(512, num_diseases))

    def forward(self, xb): # xb is the loaded batch
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

In [1]:
# defining the model and moving it to the GPU
model = to_device(ResNet9(3, len(train.classes)), device)
model

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           1,792
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
            Conv2d-4        [-1, 128, 256, 256]          73,856
       BatchNorm2d-5        [-1, 128, 256, 256]             256
              ReLU-6        [-1, 128, 256, 256]               0
         MaxPool2d-7          [-1, 128, 64, 64]               0
            Conv2d-8          [-1, 128, 64, 64]         147,584
       BatchNorm2d-9          [-1, 128, 64, 64]             256
             ReLU-10          [-1, 128, 64, 64]               0
           Conv2d-11          [-1, 128, 64, 64]         147,584
      BatchNorm2d-12          [-1, 128, 64, 64]             256
             ReLU-13          [-1, 128, 64, 64]               0
           Conv2d-14          [-1, 256,

In [ ]:
epochs = 8
max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [3]:
%%time
history += fit_OneCycle(epochs, max_lr, model, train_dl, valid_dl,
                             grad_clip=grad_clip,
                             weight_decay=1e-4,
                             opt_func=opt_func)

Epoch 1/8
48/48 [==============================] - 168s 3s/step - loss: 0.6369 - accuracy: 0.5218 - val_loss: 0.2809 - val_accuracy: 0.4864
Epoch 2/8
48/48 [==============================] - 164s 3s/step - loss: 0.1743 - accuracy: 0.5616 - val_loss: 0.2648 - val_accuracy: 0.5213
Epoch 3/8
48/48 [==============================] - 166s 3s/step - loss: 0.1570 - accuracy: 0.5898 - val_loss: 0.6024 - val_accuracy: 0.5670
Epoch 4/8
48/48 [==============================] - 164s 3s/step - loss: 0.1377 - accuracy: 0.6250 - val_loss: 0.1297 - val_accuracy: 0.6232
Epoch 5/8
48/48 [==============================] - 166s 3s/step - loss: 0.0773 - accuracy: 0.6613 - val_loss: 0.2212 - val_accuracy: 0.6064
Epoch 6/8
48/48 [==============================] - 167s 3s/step - loss: 0.0828 - accuracy: 0.7000 - val_loss: 0.1310 - val_accuracy: 0.6832
Epoch 7/8
48/48 [==============================] - 185s 4s/step - loss: 0.0505 - accuracy: 0.7837 - val_loss: 0.2200 - val_accuracy: 0.7732
Epoch 8/8
48/48 [===